In [56]:
import glob
import os

import numpy as np
import pandas as pd

In [57]:
datadir = 'F:/Documents/WETO Incubator/OpenFAST/VolturnUS15MW'
md_out_files = glob.glob(os.path.join(datadir, '*_500m.MD.Line*.out'))
numnodes = 50

In [58]:
md_out_files

['F:/Documents/WETO Incubator/OpenFAST/VolturnUS15MW\\VolturnUS-15MW_Cat_500m.MD.Line1.out',
 'F:/Documents/WETO Incubator/OpenFAST/VolturnUS15MW\\VolturnUS-15MW_Cat_500m.MD.Line2.out',
 'F:/Documents/WETO Incubator/OpenFAST/VolturnUS15MW\\VolturnUS-15MW_Cat_500m.MD.Line3.out']

In [72]:
out_dfs = []
for out_file in md_out_files:
    linenum = out_file[-5] # only valid for single-digit lines with filenames ending with '.out'
    out_df = pd.read_csv(out_file, sep='\s+', skiprows=[1], na_values=['***************'])
    out_df.columns = out_df.columns.str.replace('^Node', f'L{linenum}Node', regex=True)
    out_df.columns = out_df.columns.str.replace('^Seg', f'L{linenum}Seg', regex=True)
    out_dfs.append(out_df)


In [73]:
mdout = pd.concat(out_dfs, axis=1)

In [76]:
mdout.filter(regex='^..Node1p', axis=1)

,L1Node1px,L1Node1py,L1Node1pz,L2Node1px,L2Node1py,L2Node1pz,L3Node1px,L3Node1py,L3Node1pz
0,-1963.5306,0.000000,-500.00584,981.76528,1700.4673,-500.00584,981.76528,-1700.4673,-500.00584
1,-1963.5306,0.000000,-500.00584,981.76528,1700.4673,-500.00584,981.76528,-1700.4673,-500.00584
2,-1963.5306,0.000000,-500.00584,981.76528,1700.4673,-500.00584,981.76528,-1700.4673,-500.00584
3,-1963.5306,NaN,-500.00584,981.76528,1700.4673,-500.00584,981.76528,-1700.4673,-500.00584
4,-1963.5306,NaN,-500.00584,981.76528,1700.4673,-500.00584,981.76528,-1700.4673,-500.00584
...,...,...,...,...,...,...,...,...,...
11996,-1963.5161,-0.015710,-500.00583,982.48364,1700.0675,-500.00584,982.47868,-1700.0706,-500.00584
11997,-1963.5161,-0.015743,-500.00583,982.48206,1700.0683,-500.00584,982.47720,-1700.0714,-500.00584
11998,-1963.5161,-0.015777,-500.00583,982.48047,1700.0692,-500.00584,982.47570,-1700.0722,-500.00584
11999,-1963.5161,-0.015811,-500.00583,982.47888,1700.0701,-500.00584,982.47419,-1700.0731,-500.00584


In [29]:
maxs = {
    'Max Hor Disp': np.zeros(numnodes+1),
    'Max Vert Disp': np.zeros(numnodes+1),
    'Max Disp Mag': np.zeros(numnodes+1),
    'Max Tension': np.zeros(numnodes+1),
    'Physical Depth': np.zeros(numnodes+1),
    'Physical Span': np.zeros(numnodes+1),
}

for n in range(0, numnodes+1): # there's always a node 0 at the anchor point
    max_depth = mdout[f'Node{n}pz'][0]
    if n == 0:
        max_ten = np.nan
    else:
      max_ten = mdout[f'Seg{n}Ten'].max()
    hor_disp = np.sqrt(
        (mdout[f'Node{n}px'] - mdout[f'Node{n}px'][0])**2
      + (mdout[f'Node{n}py'] - mdout[f'Node{n}py'][0])**2
    )
    vert_disp = mdout[f'Node{n}pz'] - mdout[f'Node{n}pz'][0]
    mag_disp = np.sqrt(
        (mdout[f'Node{n}px'] - mdout[f'Node{n}px'][0])**2
      + (mdout[f'Node{n}py'] - mdout[f'Node{n}py'][0])**2
      + (mdout[f'Node{n}pz'] - mdout[f'Node{n}pz'][0])**2
    )
    max_hor_disp = hor_disp.max()
    max_vert_disp = vert_disp.max()
    max_mag_disp = mag_disp.max()

    # maxs['Partition Node'][n] = int(n)
    maxs['Max Disp Mag'][n] = max_mag_disp
    maxs['Max Hor Disp'][n] = max_hor_disp
    maxs['Max Vert Disp'][n] = max_vert_disp
    maxs['Max Tension'][n] = max_ten
    maxs['Physical Depth'][n] = max_depth


In [30]:
node_maxs = pd.DataFrame(data=maxs)

In [31]:
node_maxs

,Max Hor Disp,Max Vert Disp,Max Disp Mag,Max Tension,Physical Depth
0,0.000000,0.000000,0.000000,NaN,-500.000000
1,0.031378,0.000010,0.031378,5725394.2,-500.005840
2,0.062151,0.000000,0.062151,5725327.8,-500.005850
3,0.092120,0.000000,0.092120,5725196.2,-500.005850
4,0.121317,0.000000,0.121317,5725001.1,-500.005850
5,0.150052,0.000000,0.150052,5724745.2,-500.005850
6,0.178220,0.000000,0.178220,5724431.8,-500.005850
7,0.205470,0.000000,0.205470,5724064.8,-500.005850
8,0.231765,0.000000,0.231765,5723654.3,-500.005850
9,0.257517,0.000000,0.257517,5723228.6,-500.005850


In [48]:
partition_maxs = maxs = {
    'Max Hor Disp': np.zeros(numnodes),
    'Max Vert Disp': np.zeros(numnodes),
    'Max Disp Mag': np.zeros(numnodes),
    'Max Tension': np.zeros(numnodes),
    'Physical Depth': np.zeros(numnodes),
}
partition_nodes = np.zeros(numnodes)

In [49]:
for n in range(1, numnodes+1):
    max_phys_depth = node_maxs.loc[numnodes-n:, 'Physical Depth'].min()
    max_hor_disp = node_maxs.loc[numnodes-n:, 'Max Hor Disp'].max()
    max_vert_disp = node_maxs.loc[numnodes-n:, 'Max Vert Disp'].max()
    max_mag_disp = node_maxs.loc[numnodes-n:, 'Max Disp Mag'].max()
    max_seg_ten = node_maxs.loc[numnodes-n:, 'Max Tension'].max()

    partition_nodes[n-1] = int(n)
    partition_maxs['Max Disp Mag'][n-1] = max_mag_disp
    partition_maxs['Max Hor Disp'][n-1] = max_hor_disp
    partition_maxs['Max Vert Disp'][n-1] = max_vert_disp
    partition_maxs['Max Tension'][n-1] = max_seg_ten
    partition_maxs['Physical Depth'][n-1] = max_phys_depth

In [50]:
partition_maxs = pd.DataFrame(data=partition_maxs, index=partition_nodes)

In [51]:
partition_maxs

,Max Hor Disp,Max Vert Disp,Max Disp Mag,Max Tension,Physical Depth
1.0,47.465271,9.802840,47.660440,8500419.5,-50.807437
2.0,47.465271,14.438239,47.660440,8500419.5,-87.080089
3.0,47.465271,19.439950,47.660440,8500419.5,-122.753210
4.0,47.465271,24.554540,47.660440,8500419.5,-157.743360
5.0,47.465271,29.627060,47.660440,8500419.5,-191.955460
6.0,47.465271,34.587830,47.660440,8500419.5,-225.280610
7.0,47.465271,39.378270,47.660440,8500419.5,-257.610840
8.0,47.465271,43.929380,47.660440,8500419.5,-288.823740
9.0,47.465271,48.142170,48.158942,8500419.5,-318.764420
10.0,47.465271,51.876040,52.044838,8500419.5,-347.228970


## TODO

* Track x/y positions as well
* Max scaling at UMaine basin (30x9x4.5m)
* Scaled load/deflections